In [48]:
import pydicom as dicom
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns  # for nicer plots
sns.set(style="darkgrid")  # default style

# tf and keras
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras import metrics
tf.get_logger().setLevel('INFO')

In [49]:
import os 
from os import walk

In [50]:
# get all CN image files in subdirectories
cn_path = '/Users/adelinechin/207/20_CN_Axial'

# list to store files name
cn_list = []
for (cn_path, dir_names, file_names) in walk(cn_path):
    for file in file_names:
        if file.endswith('.dcm'):
            cn_list.append(os.path.join(cn_path, file))
print(cn_list)

['/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-T2_TSE__br_raw_20070125174800613_38_S25562_I37498.dcm', '/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-T2_TSE__br_raw_20070125174637096_8_S25562_I37498.dcm', '/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-T2_TSE__br_raw_20070125174752097_33_S25562_I37498.dcm', '/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-T2_TSE__br_raw_20070125174807457_42_S25562_I37498.dcm', '/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-T2_TSE__br_raw_20070125174645596_13_S25562_I37498.dcm', '/Users/adelinechin/207/20_CN_Axial/032_S_1169/Axial_PD-T2_TSE/2007-01-24_09_32_05.0/I37498/ADNI_032_S_1169_MR_Axial_PD-

In [51]:
# choose scans from the middle of the brain
image_id = ['25', '26', '27', '28', '29', '30']
cn_images = []
for file_name in cn_list:
    if file_name[len(file_name)-20:len(file_name)-18] in image_id:
        cn_images.append(file_name)

In [52]:
# get all AD image files in subdirectories
ad_path = '/Users/adelinechin/207/20_AD_Axial'

# list to store files name
ad_list = []
for (ad_path, dir_names, file_names) in walk(ad_path):
    for file in file_names:
        if file.endswith('.dcm'):
            ad_list.append(os.path.join(ad_path, file))
print(ad_list)

['/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-T2_TSE__br_raw_20061110153128160_11_S22331_I29299.dcm', '/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-T2_TSE__br_raw_20061110153124992_5_S22331_I29299.dcm', '/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-T2_TSE__br_raw_20061110153131500_19_S22331_I29299.dcm', '/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-T2_TSE__br_raw_20061110153140563_40_S22331_I29299.dcm', '/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-T2_TSE__br_raw_20061110153133964_26_S22331_I29299.dcm', '/Users/adelinechin/207/20_AD_Axial/067_S_0828/Axial_PD-T2_TSE/2006-11-09_11_41_47.0/I29299/ADNI_067_S_0828_MR_Axial_PD-

In [53]:
# choose scans from the middle of the brain
image_id = ['25', '26', '27', '28', '29', '30']
ad_images = []
for file_name in ad_list:
    if file_name[len(file_name)-20:len(file_name)-18] in image_id:
        ad_images.append(file_name)

In [54]:
# generate labels
cn_labels = [0]*len(cn_images)
ad_labels = [1]*len(ad_images)

In [55]:
# sanity check label lists
len(cn_labels) == len(cn_images) and len(ad_labels) == len(ad_images)

True

In [56]:
# combine CN and AD data
labels = cn_labels + ad_labels
scans = cn_images + ad_images

In [57]:
# sanity check data
len(labels) == len(scans)

True

In [58]:
# read and convert image paths to pixel arrays
images = []
for path in scans:
    ds = dicom.dcmread(path, force=True)
    try:
        # Dicom to pixel array
        pixel_array = ds.pixel_array.astype(float)
        
    except AttributeError:
        continue
    images.append(pixel_array)

In [59]:
# sanity check
len(images) == len(scans)

True

In [62]:
# list to ndarray
labels = np.ndarray((len(labels),), buffer=np.array(labels)).flatten()

for i, image in enumerate(images):
    images[i] = np.resize(image, (224, 224))
    images[i] = np.dstack([images[i], images[i], images[i]])

In [64]:
images = np.stack(images)

In [65]:
images.shape

(182, 224, 224, 3)

In [27]:
labels.shape

(182,)

### Split into train/validation/test data and apply transformations and augmentations

In [66]:
IMAGE_SIZE = (224, 224)
CONTRAST_FACTOR = 3
DELTA = 0.3

def preprocess_data_part2(images, y, split=(0.6,0.2,0.2)):
    """ Split data into train, validation and test sets; apply transformaions and augmentations

    Params:
    -------
    images  (np.ndarray): Images of shape (N, 224, 224, 3)
    y (np.ndarray): Labels of shape (N,)
    split (tuple): 3 values summing to 1 defining split of train, validation and test sets

    Returns:
    --------
    X_train (np.ndarray): Train images of shape (N_train, 224, 224, 3)
    y_train (np.ndarray): Train labels of shape (N_train,)
    X_val (np.ndarray): Val images of shape (N_val, 224, 224, 3)
    y_val (np.ndarray): Val labels of shape (N_val,)
    X_test (np.ndarray): Test images of shape (N_test, 224, 224, 3)
    y_test (np.ndarray): Test labels of shape (N_test,)

    """

    ### create train/validation/test sets ###
    #########################################
    # NOTE: Each time you run this cell, you'll re-shuffle the data. The ordering will be the same due to the random seed generator
    tf.random.set_seed(1234)
    np.random.seed(1234)
    shuffle = np.random.permutation(np.arange(images.shape[0]))
    images, y = images[shuffle], y[shuffle]

    splits = np.multiply(len(images), split).astype(int)
    X_train, X_val, X_test = np.split(images, [splits[0], splits[0]+splits[1]])
    y_train, y_val, y_test = np.split(y, [splits[0], splits[0]+splits[1]])

    ### image transformation on training, validation, and test data ###
    ###################################################################
    # image resize
    X_train = tf.image.resize(X_train, size=IMAGE_SIZE)
    X_val = tf.image.resize(X_val, size=IMAGE_SIZE)
    X_test = tf.image.resize(X_test, size=IMAGE_SIZE)

    # rescale image to [0,1], i.e., greyscale
    X_train = X_train/255.0
    X_val = X_val/255.0
    X_test = X_test/255.0


    ### image augmentation on training data ###
    ###########################################
    # adjust brightness
    X_train_augm = tf.image.adjust_brightness(X_train, delta=DELTA)

    # adjust contrast
    X_train_augm = tf.image.adjust_contrast(X_train_augm, contrast_factor=CONTRAST_FACTOR)

    # random flip
    X_train_augm = tf.image.random_flip_left_right(X_train_augm)

    # concatenate original X_train and augmented X_train data
    X_train = tf.concat([X_train, X_train_augm],axis=0)

    # concatenate y_train (note the label is preserved)
    y_train_augm = y_train
    y_train = tf.concat([y_train, y_train_augm],axis=0)

    # shuffle X_train and y_train, i.e., shuffle two tensors in the same order
    shuffle = tf.random.shuffle(tf.range(tf.shape(X_train)[0], dtype=tf.int32))
    X_train = tf.gather(X_train, shuffle)
    y_train = tf.gather(y_train, shuffle).numpy() #also transforms y_train to numpy array

    return X_train, y_train, X_val, y_val, X_test, y_test

In [67]:
X_train, y_train, X_val, y_val, X_test, y_test = preprocess_data_part2(
    images,
    labels,
    split=(0.6,0.2,0.2)
)

print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print(f"X_val shape {X_val.shape}")
print(f"y_val shape {y_val.shape}")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

X_train shape (218, 224, 224, 3)
y_train shape (218,)
X_val shape (36, 224, 224, 3)
y_val shape (36,)
X_test shape (37, 224, 224, 3)
y_test shape (37,)


### Build model

In [68]:
model = tf.keras.Sequential()

# add first convolution layer to the model
model.add(tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='same',
    data_format='channels_last',
    name='conv_1',
    activation='relu'))


# add a max pooling layer with pool size (2,2) and strides of 2
# (this will reduce the spatial dimensions by half)
model.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2),
    name='pool_1'))


# add second convolutional layer
model.add(tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=(5, 5),
    strides=(1, 1),
    padding='same',
    name='conv_2',
    activation='relu'))

# add second max pooling layer with pool size (2,2) and strides of 2
# (this will further reduce the spatial dimensions by half)
model.add(tf.keras.layers.MaxPool2D(
    pool_size=(2, 2), name='pool_2')
)


# add a fully connected layer (need to flatten the output of the previous layers first)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(
    units=1024,
    name='fc_1',
    activation='relu'))

# add dropout layer
model.add(tf.keras.layers.Dropout(
    rate=0.5))

# add the last fully connected layer
# this last layer sets the activation function to "None" in order to output the logits
# note that passing activation = "sigmoid" will return class memembership probabilities but
# in TensorFlow logits are prefered for numerical stability
# set units=1 to get a single output unit (remember it's a binary classification problem)
model.add(tf.keras.layers.Dense(
    units=1,
    name='fc_2',
    activation=None))


# build model and print summary
tf.random.set_seed(1)
model.build(input_shape=(None, 224, 224, 3))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 224, 224, 32)      2432      
                                                                 
 pool_1 (MaxPooling2D)       (None, 112, 112, 32)      0         
                                                                 
 conv_2 (Conv2D)             (None, 112, 112, 64)      51264     
                                                                 
 pool_2 (MaxPooling2D)       (None, 56, 56, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 200704)            0         
                                                                 
 fc_1 (Dense)                (None, 1024)              205521920 
                                                                 
 dropout_1 (Dropout)         (None, 1024)             

### Compile model

In [69]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), #set from_ligits=True because our last layer does not apply sigmoid
              metrics=['accuracy'])

### Fit model

In [70]:
# set random seed to get reproductible results
# neural network algorithms are stochastic (e.g., due to random weight initialization); setting a random seed helps to get more stable results after each run
# however, best way to deal with randomness is to repeat your experiment many times (30+) and use statistics to summarize the performance of the model
tf.random.set_seed(1234)
np.random.seed(1234)
history = model.fit(X_train, y_train,
                    epochs=10,
                    validation_data=(X_val, y_val)
)

Epoch 1/10
7/7 [==============================] - 25s 3s/step - loss: 0.1088 - accuracy: 0.9725 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/10
7/7 [==============================] - 20s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 20s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
7/7 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
7/7 [==============================] - 23s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
7/7 [==============================] - 22s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
7/7 [==============================] - 21s 3s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val

### Evaluation

In [71]:
test_results = model.evaluate(X_test, y_test)
print('\nTest Acc. {:.2f}%'.format(test_results[1]*100))

2/2 [==============================] - 1s 87ms/step - loss: 0.0000e+00 - accuracy: 1.0000

Test Acc. 100.00%
